<a href="https://colab.research.google.com/github/Amasha03/Customer_Churn_Prediction/blob/main/notebooks/04_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Telco Customer Churn Prediction
### CM2604 Machine Learning CourseWork
04-Neural Network

Student Name : Amasha Widanagamage  
RGU ID : 2425800  
IIT ID : 20241246

In [87]:
pip install keras-tuner --upgrade


In [88]:
#Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import tensorflow as tf
from tensorflow.keras import layers, models
import keras_tuner as kt

In [89]:
#Load dataset
df = pd.read_csv('/content/drive/MyDrive/ML/coursework/preprocessed_data.csv')


In [90]:
#Feature Selection

selected_features = ['tenure',
    'MonthlyCharges',
    'TotalCharges',
    'PaperlessBilling',
    'Contract_Two year',
    'Contract_One year',
    'InternetService_Fiber optic',
    'OnlineBackup_Yes',
    'TechSupport_No',
    'PaymentMethod_Electronic check',
    'OnlineSecurity_Yes',
    'OnlineSecurity_No',
    'StreamingTV_No',
    'SeniorCitizen',]

X = df[selected_features]
y = df['Churn']

In [91]:
#Test/Train Split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [92]:
#Scale features

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [93]:
# Build default neural network
model_default = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

model_default.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [94]:
# Train Model
history_default = model_default.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=80,
    batch_size=32,
    verbose=1
)

Epoch 1/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.7340 - auc_1: 0.7418 - loss: 0.5170 - val_accuracy: 0.7982 - val_auc_1: 0.8472 - val_loss: 0.4120
Epoch 2/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8017 - auc_1: 0.8306 - loss: 0.4296 - val_accuracy: 0.7964 - val_auc_1: 0.8480 - val_loss: 0.4108
Epoch 3/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7991 - auc_1: 0.8397 - loss: 0.4236 - val_accuracy: 0.7982 - val_auc_1: 0.8500 - val_loss: 0.4108
Epoch 4/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7994 - auc_1: 0.8363 - loss: 0.4319 - val_accuracy: 0.7991 - val_auc_1: 0.8515 - val_loss: 0.4140
Epoch 5/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8009 - auc_1: 0.8472 - loss: 0.4264 - val_accuracy: 0.7947 - val_auc_1: 0.8502 - val_loss: 0.4100
Epoch 6/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7923 - auc_1: 0.8460 - loss: 0.4237 - val_accuracy: 0.7938 - val_auc_1: 0.8480 - val_loss: 0.4172
Epoch 7/80


In [95]:
# Evaluate
loss, acc, auc = model_default.evaluate(X_test, y_test)
print(f"Default NN Accuracy: {acc*100:.2f}%")
print(f"Default NN ROC-AUC: {auc*100:.2f}%")


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7700 - auc_1: 0.7981 - loss: 0.4900
Default NN Accuracy: 78.04%
Default NN ROC-AUC: 80.32%


In [96]:
#Model function for Keras tuner

def build_model(hp):
    model = models.Sequential()
    # First layer
    model.add(layers.Dense(
        units=hp.Int('units1', min_value=128, max_value=256, step=32),
        activation='relu',
        input_shape=(X_train.shape[1],)
    ))
    model.add(layers.Dropout(rate=hp.Float('dropout1', 0.2, 0.5, step=0.1)))

    # Second layer
    model.add(layers.Dense(
        units=hp.Int('units2', min_value=64, max_value=128, step=16),
        activation='relu'
    ))
    model.add(layers.Dropout(rate=hp.Float('dropout2', 0.2, 0.5, step=0.1)))

    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC()]
    )

    return model

In [97]:
#Initialize RandomSearch Tuner

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='telco_nn_tuning'
)

Reloading Tuner from tuner_dir/telco_nn_tuning/tuner0.json


In [98]:
#Run Hyperparameter Tuning

tuner.search(X_train,
             y_train,
             validation_split=0.2,
             epochs=80,
             batch_size=32,
             verbose=1)

In [99]:
#Get Best Model

best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

test_loss, test_acc, test_auc = best_model.evaluate(X_test, y_test)

print("Best Hyperparameters:", best_hp.values)

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7829 - auc: 0.8332 - loss: 0.4307
Best Hyperparameters: {'units1': 224, 'dropout1': 0.30000000000000004, 'units2': 96, 'dropout2': 0.2, 'learning_rate': 0.001}


In [100]:
#Evaluate on Test Set

test_loss, test_acc, test_auc = best_model.evaluate(X_test, y_test)
print("\n\nTuned Model")
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test AUC: {test_auc*100:.2f}%")

loss, acc, auc = model_default.evaluate(X_test, y_test)
print("\n\nDefault Model")
print(f"Default NN Accuracy: {acc*100:.2f}%")
print(f"Default NN ROC-AUC: {auc*100:.2f}%")

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7829 - auc: 0.8332 - loss: 0.4307


Tuned Model
Test Accuracy: 79.18%
Test AUC: 82.98%
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7700 - auc_1: 0.7981 - loss: 0.4900


Default Model
Default NN Accuracy: 78.04%
Default NN ROC-AUC: 80.32%


In [101]:
# Predictions
y_pred = (best_model.predict(X_test) > 0.5).astype("int32")
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

y_pred=(model_default.predict(X_test) > 0.5).astype("int32")
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

Confusion Matrix:
 [[905 128]
 [165 209]]

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.88      0.86      1033
           1       0.62      0.56      0.59       374

    accuracy                           0.79      1407
   macro avg       0.73      0.72      0.72      1407
weighted avg       0.79      0.79      0.79      1407

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

Confusion Matrix:
 [[894 139]
 [170 204]]

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.87      0.85      1033
           1       0.59      0.55      0.57       374

    accuracy                           0.78      1407
   macro avg       0.72      0.71      0.71      1407
weighted avg       0.77      0.78      0.78      1407

